In [17]:
import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel

![alternatvie text](2023-06-30-22-20-21.png)  

With minimum number of colors, color the nodes such that nodes connected to each other have different colors

In [3]:
arcs= [(0,1),(1,2),(2,3),(3,4),(0,4),(1,5),(0,6),(2,7),(3,8),(4,9),(5,9),(5,8),(6,7),(6,8),(7,9)]

Nc = 5
N = 10

In [46]:
def binary_var_series(df, mdl,**kargs):
    return pd.Series(mdl.binary_var_list(len(df), **kargs), index = df.index)
def integer_var_series(df, mdl,**kargs):
    return pd.Series(mdl.integer_var_list(len(df), **kargs), index = df.index)


def extract_solution(msol, df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [msol.get_value(dvar)   for dvar in df[xDVarName]]  #dvar.solution_value
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df    

In [105]:
mdl = CpoModel(name="GraphColoring")

Colors = pd.DataFrame(np.arange(0,Nc).reshape(-1,1), columns = ["ColorId"]).set_index("ColorId")
Colors['u']= binary_var_series(Colors, mdl, name = 'u')
display(Colors)

NodeColors = pd.DataFrame(np.arange(0,N).reshape(-1,1), columns = ["NodeId"]).set_index("NodeId")
NodeColors['X']= integer_var_series(NodeColors, mdl, domain =np.arange(0,Nc),  name = 'x')
display(NodeColors)



,u
ColorId,
0,"u_0 = intVar(0, 1)"
1,"u_1 = intVar(0, 1)"
2,"u_2 = intVar(0, 1)"
3,"u_3 = intVar(0, 1)"
4,"u_4 = intVar(0, 1)"


,X
NodeId,
0,x_0 = intVar(0..4)
1,x_1 = intVar(0..4)
2,x_2 = intVar(0..4)
3,x_3 = intVar(0..4)
4,x_4 = intVar(0..4)
5,x_5 = intVar(0..4)
6,x_6 = intVar(0..4)
7,x_7 = intVar(0..4)
8,x_8 = intVar(0..4)


In [106]:
for (i,j) in arcs:
    df = NodeColors.reset_index(drop = False)
    mdl.add (df[df['NodeId']==i]['X'].values[0] != df[df['NodeId']==j]['X'].values[0] )


In [107]:
# use minimum number of colors

Colors  = Colors.reset_index(drop = False)
Colors["expressions"] = Colors["ColorId"]  * Colors["u"] 
display(Colors)

ColorCosts = mdl.sum(Colors["expressions"])
mdl.add_kpi(ColorCosts, "ColorCosts")
ColorCosts


,ColorId,u,expressions
0,0,"u_0 = intVar(0, 1)",0 * u_0
1,1,"u_1 = intVar(0, 1)",1 * u_1
2,2,"u_2 = intVar(0, 1)",2 * u_2
3,3,"u_3 = intVar(0, 1)",3 * u_3
4,4,"u_4 = intVar(0, 1)",4 * u_4


In [115]:
mdl.minimize(ColorCosts)

msol = mdl.solve( log_output = None)
if msol.is_solution():
    a= extract_solution(msol,NodeColors, extract_dvar_names=['X'])

a
    

,X_Solution
NodeId,
0,2
1,1
2,2
3,0
4,1
5,0
6,0
7,1
8,1


In [119]:
msol = mdl.solve( log_output = None)
if msol.is_solution():
    a= extract_solution(msol,Colors, extract_dvar_names=['u'])

a

,ColorId,expressions,u_Solution
0,0,0 * u_0,0
1,1,1 * u_1,0
2,2,2 * u_2,0
3,3,3 * u_3,0
4,4,4 * u_4,0


In [131]:

msol.get_value(a['expressions'].iloc[0])

KeyError: "Variable or KPI '0 * u_0' not in the solution"

In [130]:
a['expressions'].iloc[0].pretty_print()

times(
 | 0
 | u_0)

In [129]:
ColorCosts

sum(
 | [
 |  | times(
 |  |  | 0
 |  |  | u_0)
 |  | times(
 |  |  | 1
 |  |  | u_1)
 |  | times(
 |  |  | 2
 |  |  | u_2)
 |  | times(
 |  |  | 3
 |  |  | u_3)
 |  | times(
 |  |  | 4
 |  |  | u_4)])